In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
;; Use full paths for ACL2-MCP compatibility
;; Trust tags needed for quicklisp/HTTP client
(include-book "/workspaces/acl2-swf-experiments/experiments/agents/verified-agent")
(include-book "/workspaces/acl2-swf-experiments/experiments/agents/llm-client"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) (:http-json) (:llm-client)))

Note (from clause-processors/equality): disabling DISJOIN, DISJOIN2,
CONJOIN and CONJOIN2.


Summary
Form:  ( INCLUDE-BOOK 
"/workspaces/acl2-swf-experiments/experiments/agents/verified-agent"
...)
Rules: NIL
Time:  0.83 seconds (prove: 0.00, print: 0.00, other: 0.83)
 "/workspaces/acl2-swf-experiments/experiments/agents/verified-agent.lisp"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

TTAG NOTE (for included book): Adding ttag :HTTP-JSON from book /workspaces/acl2-swf-experiments/experiments/agents/http-json.

TTAG NOTE (for included book): Adding ttag :LLM-CLIENT from book /workspaces/acl2-swf-experiments/experiments/agents/llm-client.

Summary
Form:  ( INCLUDE-BOOK 
"/workspaces/acl2-swf-experiments/experiments/agents/llm-client" ...)
Rules: NIL
Time:  0.39 seconds (prove: 0.00, print: 0.00, other: 0.39)
 "/workspaces/acl2

In [3]:
;; Model preferences in order - first match wins
(defconst *model-prefs* '("qwen" "nemotron" "llama" "gemma"))


Summary
Form:  ( DEFCONST *MODEL-PREFS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-PREFS*


In [4]:
;; Get full model info and select best loaded completions model
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (prog2$ 
    (if err
        (cw "~%Error getting models: ~s0~%" err)
      (progn$
       (cw "~%Available models: ~x0~%" (len models))
       (cw "Loaded completions models: ~x0~%" 
           (len (filter-loaded-completions-models models)))))
    (mv nil '(value-triple :models-listed) state))))


Available models: 14
Loaded completions models: 3

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
 :MODELS-LISTED


In [5]:
;; Select the best model based on preferences
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (prog2$
      (if selected
          (cw "~%Selected model: ~s0 (context: ~x1 tokens)~%"
              (model-info->id selected)
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No suitable model found! Make sure LM Studio is running.~%"))
      (mv nil 
          `(defconst *model-id* 
             ,(if selected (model-info->id selected) "no-model-found"))
          state)))))


Selected model: qwen/qwen3-coder-30b (context: 80000 tokens)

Summary
Form:  ( DEFCONST *MODEL-ID* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-ID*


In [6]:
(defconst *initial-state*
  (make-agent-state 
    :max-steps 20              ; Allow up to 20 conversation turns
    :token-budget 50000        ; Token budget for tool calls
    :time-budget 3600          ; 1 hour time budget
    :file-access 1             ; Read access to files
    :execute-allowed nil       ; No code execution
    :max-context-tokens 8000   ; Context window size
    :satisfaction 0))          ; Starting satisfaction


Summary
Form:  ( DEFCONST *INITIAL-STATE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *INITIAL-STATE*


In [7]:
(defconst *system-prompt*
  "You are a helpful AI assistant running inside a formally verified agent framework built in ACL2. 
You can engage in conversation and help with questions.
Be concise but helpful in your responses.")


Summary
Form:  ( DEFCONST *SYSTEM-PROMPT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SYSTEM-PROMPT*


In [8]:
(defconst *agent-v1*
  (init-agent-conversation *system-prompt* *initial-state*))


Summary
Form:  ( DEFCONST *AGENT-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V1*


In [9]:
;; Verify the conversation was initialized
#-skip-interactive
(defconst *check-init*
  (prog2$ (cw "Agent initialized. Messages: ~x0~%" 
              (len (get-messages *agent-v1*)))
          t))

Agent initialized. Messages: 1

Summary
Form:  ( DEFCONST *CHECK-INIT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHECK-INIT*


In [10]:
(defun show-messages (msgs)
  (declare (xargs :mode :program))
  (if (endp msgs)
      nil
    (let* ((msg (car msgs))
           (role (chat-message->role msg))
           (content (chat-message->content msg))
           (role-str (chat-role-case role
                       :system "SYSTEM"
                       :user "USER"
                       :assistant "ASSISTANT"
                       :tool "TOOL")))
      (prog2$ (cw "~%[~s0]~%~s1~%" role-str content)
              (show-messages (cdr msgs))))))


Summary
Form:  ( DEFUN SHOW-MESSAGES ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-MESSAGES


In [11]:
(defun show-conversation (st)
  (declare (xargs :mode :program))
  (prog2$ (cw "~%========== Conversation ==========")
          (prog2$ (show-messages (get-messages st))
                  (cw "~%===================================~%"))))


Summary
Form:  ( DEFUN SHOW-CONVERSATION ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-CONVERSATION


In [12]:
#-skip-interactive
(defconst *agent-v2*
  (add-user-msg "Hello! Can you explain what a formally verified agent is?" 
                *agent-v1*))


Summary
Form:  ( DEFCONST *AGENT-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V2*


In [13]:
;; Show the conversation so far
#-skip-interactive
(defconst *show-v2*
  (show-conversation *agent-v2*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?


Summary
Form:  ( DEFCONST *SHOW-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V2*


In [14]:
;; This calls the LLM and adds the response to conversation
;; Requires LM Studio running!
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v2*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *agent-v3* *agent-v2*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *agent-v3* 
                        (add-assistant-msg ,response *agent-v2*)) 
                 state)))))


Assistant: A formally verified agent is an AI system whose behavior
and decision-making processes have been mathematically proven to satisfy
specific properties or constraints. In the context of ACL2 (A Computational
Logic for Applicative Common Lisp), this means:

1. **Mathematical Proofs**: The agent's logic and algorithms are formally
proved correct using rigorous mathematical techniques within the ACL2
theorem prover framework.

2. **Specified Properties**: The agent's behavior is defined by precise
specifications that can be verified, such as safety guarantees, correctness
conditions, or desired performance bounds.

3. **Verification Framework**: ACL2 provides the tools to write these
specifications and automatically check that the agent implementation
meets them.

4. **Reliability**: Unlike traditional software testing, formal verification
provides guarantees that the agent will behave correctly under all
conditions specified in its formal specification.

In practice, this means

In [15]:
;; Show updated conversation
#-skip-interactive
(defconst *show-v3*
  (show-conversation *agent-v3*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or constraints. In the context of ACL2 (A Computational
Logic for Applicative Common Lisp), this means:

1. **Mathematical Proofs**: The agent's logic and algorithms are formally
proved correct using rigorous mathematical techniques within the ACL2
theorem prover framework.

2. **Specified Properties**: The agent's behavior is defined by precise
specifications that can be verified, such as safety guarantees, correctness
conditions, or desired performance bounds.

3. **Verification Framework**: ACL2 provides the tools to wri

In [16]:
#-skip-interactive
(defconst *agent-v4*
  (add-user-msg "What properties have been proven about this agent?" 
                *agent-v3*))


Summary
Form:  ( DEFCONST *AGENT-V4* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V4*


In [17]:
#-skip-interactive
(defconst *show-v4*
  (show-conversation *agent-v4*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or constraints. In the context of ACL2 (A Computational
Logic for Applicative Common Lisp), this means:

1. **Mathematical Proofs**: The agent's logic and algorithms are formally
proved correct using rigorous mathematical techniques within the ACL2
theorem prover framework.

2. **Specified Properties**: The agent's behavior is defined by precise
specifications that can be verified, such as safety guarantees, correctness
conditions, or desired performance bounds.

3. **Verification Framework**: ACL2 provides the tools to wri

In [18]:
;; Call LLM and add response to conversation
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v4*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *agent-v5* *agent-v4*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *agent-v5* 
                        (add-assistant-msg ,response *agent-v4*)) 
                 state)))))


Assistant: As an AI assistant running inside a formally verified agent
framework built in ACL2, I can tell you that the properties that have
been formally proven about this agent include:

1. **Correctness of Implementation**: The agent's algorithms and logic
have been mathematically proven to correctly implement the intended
specifications.

2. **Safety Properties**: The agent's behavior adheres to safety constraints
- it won't perform actions that violate predefined safety boundaries.

3. **Consistency**: The agent maintains internal consistency in its
reasoning and responses over time.

4. **Specification Compliance**: All responses and actions are guaranteed
to conform to the formal specifications that define its behavior.

5. **Reasoning Soundness**: The logical inference processes used by
the agent are sound, meaning if the agent derives a conclusion, it
is logically valid given its premises.

However, I should note that as an AI assistant within this framework,
I don't have acc

In [19]:
#-skip-interactive
(defconst *show-v5*
  (show-conversation *agent-v5*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or constraints. In the context of ACL2 (A Computational
Logic for Applicative Common Lisp), this means:

1. **Mathematical Proofs**: The agent's logic and algorithms are formally
proved correct using rigorous mathematical techniques within the ACL2
theorem prover framework.

2. **Specified Properties**: The agent's behavior is defined by precise
specifications that can be verified, such as safety guarantees, correctness
conditions, or desired performance bounds.

3. **Verification Framework**: ACL2 provides the tools to wri

In [20]:
#-skip-interactive
(defconst *agent-v6*
  (add-user-msg "That's impressive! How does the context management work?" 
                *agent-v5*))


Summary
Form:  ( DEFCONST *AGENT-V6* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V6*


In [21]:
;; Call LLM for Turn 3
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v6*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *agent-v7* *agent-v6*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *agent-v7* 
                        (add-assistant-msg ,response *agent-v6*)) 
                 state)))))


Assistant: In a formally verified agent framework like ACL2, context
management works through formal specifications that ensure the agent
maintains consistent state and reasoning across interactions.

The key aspects of context management in such systems include:

1. **State Representation**: The agent's context (including conversation
history, user preferences, and environmental information) is formally
represented using data structures that can be mathematically reasoned
about.

2. **Context Preservation**: Formal proofs ensure that the agent maintains
context consistency - for example, that previous conversations don't
contradict current reasoning or that state transitions are valid.

3. **Scope Management**: The system formally verifies that context
is properly scoped - what information is available at different points
in the conversation and when it should be updated or discarded.

4. **Consistency Checking**: The framework automatically checks that
context updates don't introduc

In [22]:
#-skip-interactive
(defconst *show-v7*
  (show-conversation *agent-v7*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or constraints. In the context of ACL2 (A Computational
Logic for Applicative Common Lisp), this means:

1. **Mathematical Proofs**: The agent's logic and algorithms are formally
proved correct using rigorous mathematical techniques within the ACL2
theorem prover framework.

2. **Specified Properties**: The agent's behavior is defined by precise
specifications that can be verified, such as safety guarantees, correctness
conditions, or desired performance bounds.

3. **Verification Framework**: ACL2 provides the tools to wri

In [23]:
#-skip-interactive
(defconst *context-check*
  (let* ((msgs (get-messages *agent-v7*))
         (char-len (messages-char-length msgs))
         (token-est (messages-token-estimate msgs))
         (max-tokens (agent-state->max-context-tokens *agent-v7*)))
    (prog2$ (cw "~%Context Usage:~%")
      (prog2$ (cw "  Total characters: ~x0~%" char-len)
        (prog2$ (cw "  Estimated tokens: ~x0~%" token-est)
          (prog2$ (cw "  Max tokens: ~x0~%" max-tokens)
            (prog2$ (cw "  Safety margin: ~x0~%" *context-safety-margin*)
              (cw "  Fits in context: ~x0~%" 
                  (messages-fit-p msgs max-tokens)))))))))


Context Usage:
  Total characters: 4789
  Estimated tokens: 1198
  Max tokens: 8000
  Safety margin: 500
  Fits in context: T

Summary
Form:  ( DEFCONST *CONTEXT-CHECK* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CONTEXT-CHECK*


In [24]:
;; Simulate calling a file-read tool
#-skip-interactive
(defconst *agent-with-tool*
  (add-tool-result 
    "File contents of verified-agent.lisp (first 500 chars):
; Verified ReAct Agent - ACL2 Implementation
; This file implements a formally verified ReAct agent...
(in-package \"ACL2\")
(include-book \"std/util/define\" :dir :system)
..." 
    *agent-v7*))


Summary
Form:  ( DEFCONST *AGENT-WITH-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-WITH-TOOL*


In [25]:
#-skip-interactive
(defconst *show-tool*
  (show-conversation *agent-with-tool*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified agent
framework built in ACL2.  You can engage in conversation and help with
questions. Be concise but helpful in your responses.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or constraints. In the context of ACL2 (A Computational
Logic for Applicative Common Lisp), this means:

1. **Mathematical Proofs**: The agent's logic and algorithms are formally
proved correct using rigorous mathematical techniques within the ACL2
theorem prover framework.

2. **Specified Properties**: The agent's behavior is defined by precise
specifications that can be verified, such as safety guarantees, correctness
conditions, or desired performance bounds.

3. **Verification Framework**: ACL2 provides the tools to wri

In [26]:
#-skip-interactive
(defun chat-turn (user-msg agent-st model-id state)
  "Execute one chat turn: add user message, call LLM, add response"
  (declare (xargs :mode :program :stobjs state))
  (let ((st-with-user (add-user-msg user-msg agent-st)))
    (mv-let (err response state)
      (llm-chat-completion model-id (get-messages st-with-user) state)
      (if err
          (prog2$ (cw "~%Error: ~s0~%" err)
                  (mv st-with-user state))
        (let ((st-with-response (add-assistant-msg response st-with-user)))
          (prog2$ (cw "~%Assistant: ~s0~%" response)
                  (mv st-with-response state)))))))


Summary
Form:  ( DEFUN CHAT-TURN ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT-TURN


In [27]:
;; Create a tool spec for demonstration
#-skip-interactive
(defconst *read-file-tool*
  (make-tool-spec
    :name 'read-file
    :required-access 1      ; Needs read access
    :requires-execute nil
    :token-cost 100
    :time-cost 5))


Summary
Form:  ( DEFCONST *READ-FILE-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *READ-FILE-TOOL*


In [28]:
;; Check if we can invoke this tool
#-skip-interactive
(defconst *can-read-file*
  (prog2$ (cw "~%Can invoke read-file tool: ~x0~%" 
              (can-invoke-tool-p *read-file-tool* *agent-v7*))
          (can-invoke-tool-p *read-file-tool* *agent-v7*)))


Can invoke read-file tool: T

Summary
Form:  ( DEFCONST *CAN-READ-FILE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CAN-READ-FILE*


In [29]:
;; Simulate a ReAct step with conversation
#-skip-interactive
(defconst *agent-after-react*
  (if (and (not (must-respond-p *agent-v7*))
           (can-invoke-tool-p *read-file-tool* *agent-v7*))
      (react-step-with-conversation
        (agent-action-tool-call 'read-file "verified-agent.lisp")
        *read-file-tool*
        "I'll read the verified-agent.lisp file to show you the implementation."
        "File read successfully. Contents: (in-package ACL2)..."
        *agent-v7*)
    *agent-v7*))


Summary
Form:  ( DEFCONST *AGENT-AFTER-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-AFTER-REACT*


In [30]:
#-skip-interactive
(defconst *show-react*
  (prog2$ (cw "~%After ReAct step:~%")
    (prog2$ (cw "  Step counter: ~x0~%" 
                (agent-state->step-counter *agent-after-react*))
      (prog2$ (cw "  Token budget: ~x0~%" 
                  (agent-state->token-budget *agent-after-react*))
        (cw "  Message count: ~x0~%" 
            (len (get-messages *agent-after-react*)))))))


After ReAct step:
  Step counter: 1
  Token budget: 49900
  Message count: 9

Summary
Form:  ( DEFCONST *SHOW-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-REACT*


In [31]:
#-skip-interactive
(defconst *demo-complete*
  (prog2$ (cw "~%~%========================================~%")
    (prog2$ (cw "Chat Demo Complete!~%")
      (prog2$ (cw "========================================~%")
        (prog2$ (cw "~%Key points demonstrated:~%")
          (prog2$ (cw "  1. Agent state initialization~%")
            (prog2$ (cw "  2. Conversation management (add-user-msg, add-assistant-msg)~%")
              (prog2$ (cw "  3. Context length tracking~%")
                (prog2$ (cw "  4. Tool result handling~%")
                  (prog2$ (cw "  5. ReAct step with conversation~%")
                    (prog2$ (cw "  6. State preservation verification~%")
                      (cw "~%For live LLM chat, ensure LM Studio is running at:~%  http://host.docker.internal:1234~%"))))))))))))



Chat Demo Complete!

Key points demonstrated:
  1. Agent state initialization
  2. Conversation management (add-user-msg, add-assistant-msg)
  3. Context length tracking
  4. Tool result handling
  5. ReAct step with conversation
  6. State preservation verification

For live LLM chat, ensure LM Studio is running at:
  http://host.docker.internal:1234

Summary
Form:  ( DEFCONST *DEMO-COMPLETE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-COMPLETE*
